# Ptolemy's *Geography*

This notebook loads a TEI edition of Ptolemy's *Geography*, and extracts a database of points with longitude-latitude locations from it.


## Notebook configuration

In [1]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::ptolemy:1.2.1`

import $ivy.$                                  

## Retrieve and parse TEI edition

In [3]:
import scala.xml._

val url = "https://raw.githubusercontent.com/neelsmith/ptolemy/master/tei/tlg0363.tlg009.epist03-p5-u8.xml"
val root = XML.load(url)

import scala.xml._


url: String = "https://raw.githubusercontent.com/neelsmith/ptolemy/master/tei/tlg0363.tlg009.epist03-p5-u8.xml"
root: Elem = <TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Ptolemy, Geography</title>
            </titleStmt>
            <publicationStmt>
                <p>E-text</p>
            </publicationStmt>
            <sourceDesc>
                <p>Composite based on print editions of Müller and Nobbe.</p>
            </sourceDesc>
        </fileDesc>
        <profileDesc>
            <langUsage>
                <language ident="grc">Greek</language>
                <language ident="eng">English</language>
            </langUsage>
        </profileDesc>
    </teiHeader>
    <text>
        <body xml:lang="grc">


            <interpGrp type="continents">
                <interp xml:id="Europe"/>
                <interp xml:id="Asia"/>
                <interp xml:id="Libya"/>
       

In [10]:
import edu.holycross.shot.ptolemy._

val delimited = TeiParser.parseTEI(root, false)
val ptolStrings = delimited.map(ln => PtolemyString(ln)).filter(_.id.nonEmpty)


2020-02-21 10:36:59.677Z error [TeiParser] No name element in   - (TeiParser.scala:98)
2020-02-21 10:36:59.680Z error [TeiParser] No name element in   - (TeiParser.scala:98)
2020-02-21 10:36:59.681Z error [TeiParser] No name element in   - (TeiParser.scala:98)
2020-02-21 10:36:59.681Z error [TeiParser] No name element in   - (TeiParser.scala:98)


import edu.holycross.shot.ptolemy._


delimited: Vector[String] = Vector(
  "2.2.1#Europe#hibernia#paralios#pt_ll_1#\u0392\u1f79\u03c1\u03b5\u03b9\u03bf\u...
ptolStrings: Vector[PtolemyString] = Vector(
  PtolemyString(
...

## Precision

Collect fractional values to infer implied precision.


In [11]:
val lonPrecision = ptolStrings.groupBy( pt => pt.lonFract).toVector.sortBy(_._1).map{ case (k,v) => (k,v.size)}
val latPrecision = ptolStrings.groupBy( pt => pt.latFract).toVector.sortBy(_._1).map{ case (k,v) => (k,v.size)}

lonPrecision: Vector[(String, Int)] = Vector(
  ("0.0", 2107),
...
latPrecision: Vector[(String, Int)] = Vector(
  ("0.0", 1326),
...

Configure `plotly` library to visualize:

In [6]:
// Make plotly libraries available to this notebook:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

import $ivy.$                                      

In [7]:
// Import plotly libraries, and set display defaults suggested for use in Jupyter NBs:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


In [15]:
val fractionLabels = 
Map ("0.0" -> "0", 
     "0.083" -> "1/12", 
     "0.167" -> "1/6", 
     "0.25" -> "1/4", 
     "0.333" -> "1/3", 
     "0.417" -> "5/12", 
     "0.5" -> "1/2", 
     "0.583" -> "7/12", 
     "0.667" -> "2/3", 
     "0.75" -> "3/4", 
     "0.833" -> "5/6", 
     "0.917" -> "11/12")


Vector(0.0, 0.083, 0.167, 0.25, 0.333, 0.417, 0.5, 0.583, 0.667, 0.75, 0.833, 0.917)


In [ ]:

val lonValues = lonPrecision.map(freq => fractionLabels(freq._1))
val lonCounts = lonPrecision.map(freq => freq._2)

val lonFreqs = Vector(
  Bar(x = lonValues, y = lonCounts)
)
plot(lonFreqs)

In [ ]:
/*
val latValues = latPrecision.map(freq => freq._1)
val latCounts = latPrecision.map(freq => freq._2)

val latFreqs = Vector(
  Bar(x = latValues, y = latCounts)
)
plot(latFreqs)
*/